1.  One_hot_encoding for nominal data 
2.  Decode Employment Occupation with WEight of Evidence (WOE)
3.  Decoding Ordinal string Data to ordinal string Data 
4.  Impute Missing Values with Mean 
5.  Rename Colums
6.  Check for Multicoliniarity -> Drop Columns
7.  Balance Dataset
8.  Save Version of datasets
9.  Standardize Datasets 
10. Save Versions 



In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import imblearn
from imblearn.over_sampling import SMOTE

In [2]:
data_path = Path.cwd().parent.parent / "data" / "raw"
raw_df = pd.read_csv(data_path / "training_set_features.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path / "training_set_labels.csv", index_col="respondent_id")
test_df = pd.read_csv(data_path / "test_set_features.csv", index_col="respondent_id")

all_df = raw_df.join(labels_df)


In [3]:
pd.set_option("display.max_rows", 20, "display.max_columns", 20)

In [4]:
# one-hot-enconding without employment_occuptation (employment_occuptation -> Weight of Evidence Encoded)
    # drop column employment_occupation 
    # identify columns with categorial data 
    # just use columns with nominal data
    # getdummies() -> drop first column to dismiss multicollinarity, keep NaN column
raw_df = raw_df.drop(columns = ['employment_occupation'])

non_numerical_obj = raw_df.columns[raw_df.dtypes == "object"]
non_numerical_obj

nominal_list = ['race', 'sex', 
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry']

raw_df_encoded = pd.get_dummies(raw_df, columns=nominal_list, drop_first = True, dummy_na=True)
test_df_encoded = pd.get_dummies(test_df, columns=nominal_list, drop_first = True, dummy_na=True) 

In [5]:
# Weight of Evidence Encoding of Occuptation on raw_data
# 'h1n1_vaccine', 'seasonal_vaccine' 
def WOE(var, tar):
    all_df[var] = all_df[var].fillna('NoData')
    k = all_df[[var,tar]].groupby(var)[tar].agg(['count','sum']).reset_index()
    k.columns = [var,'Count','Good']
    k['Bad'] = k['Count'] - k['Good']
    k['Good %'] = (k['Good'] / k['Good'].sum()*100).round(2)
    k['Bad %'] = (k['Bad'] / k['Bad'].sum()*100).round(2)
    k[var+tar+'_WOE'] = np.log(k['Good %'] / k['Bad %']).round(2)
    k = k.sort_values(by=var+tar+'_WOE')
    return(k)
h1n1_WOE = WOE('employment_occupation' , 'h1n1_vaccine')
seasonal_WOE = WOE('employment_occupation' , 'seasonal_vaccine')

WOE_df_season = pd.merge(all_df[['seasonal_vaccine','employment_occupation']],seasonal_WOE[['employment_occupation','employment_occupationseasonal_vaccine_WOE']],
     left_on='employment_occupation',
     right_on='employment_occupation',how='left')
WOE_df_h1n1 = pd.merge(all_df[['h1n1_vaccine','employment_occupation']],h1n1_WOE[['employment_occupation','employment_occupationh1n1_vaccine_WOE']],
     left_on='employment_occupation',
     right_on='employment_occupation',how='left')


WOE_df_both = WOE_df_h1n1.join(WOE_df_season, lsuffix='_h1n1', rsuffix='_seasonal')
WOE_mean = WOE_df_both[['employment_occupationh1n1_vaccine_WOE', 
                           'employment_occupationseasonal_vaccine_WOE']].mean(axis = 1)

raw_df_encoded['employment_occupation_WOE'] = WOE_mean

In [6]:
# Weight of Evidence Encoding of Occuptation on test_data
# 'h1n1_vaccine', 'seasonal_vaccine' 
test_df_encoded['employment_occupation_new'] =test_df_encoded['employment_occupation'].replace(np.nan, 'NoData')

test_WOE_df_season = pd.merge(test_df_encoded['employment_occupation_new'],seasonal_WOE[['employment_occupation','employment_occupationseasonal_vaccine_WOE']],
     left_on='employment_occupation_new',
     right_on='employment_occupation',how='left')
test_WOE_df_season.index =  test_df_encoded.index

test_WOE_df_h1n1 = pd.merge(test_df_encoded[['employment_occupation_new']],h1n1_WOE[['employment_occupation','employment_occupationh1n1_vaccine_WOE']],
     left_on='employment_occupation_new',
     right_on='employment_occupation',how='left')
test_WOE_df_h1n1.index =  test_df_encoded.index


test_WOE_df_both = test_WOE_df_h1n1 .join(test_WOE_df_season, lsuffix='_h1n1', rsuffix='_seasonal')

test_WOE_df_both.index =  test_df_encoded.index


test_df_encoded['employment_occupation_WOE'] = test_WOE_df_both[['employment_occupationh1n1_vaccine_WOE', 
                           'employment_occupationseasonal_vaccine_WOE']].mean(axis = 1)



test_df_encoded = test_df_encoded.drop(columns = ['employment_occupation','employment_occupation_new' ])




In [7]:
# encoding of ordinal data 
    # identifiying ordinal data columns 
    # change NaN in ordinal data columns to most frequent value 
    # change strings in ordinal data to numbers 
    # delete values in columns wich where imputed with most frequent values
    # impute NaN values with KNN
    
ordinal_list = ['age_group','education', 'income_poverty' ]



In [8]:
# impute misssing values in ordinal data columns with most frequent value 


from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
raw_df_encoded_mode = imp_mode.fit_transform(raw_df_encoded[ordinal_list])
raw_df_encoded_mode_pd = pd.DataFrame(raw_df_encoded_mode, index = raw_df_encoded.index)

raw_df_encoded['age_group'] = raw_df_encoded_mode_pd[0]
raw_df_encoded['education'] = raw_df_encoded_mode_pd[1]
raw_df_encoded['income_poverty'] = raw_df_encoded_mode_pd[2]

test_imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
test_df_encoded_mode = test_imp_mode.fit_transform(test_df_encoded[ordinal_list])
test_df_encoded_mode_pd = pd.DataFrame(test_df_encoded_mode, index = test_df_encoded.index )

test_df_encoded['age_group'] = test_df_encoded_mode_pd[0]
test_df_encoded['education'] = test_df_encoded_mode_pd[1]
test_df_encoded['income_poverty'] = test_df_encoded_mode_pd[2]

In [9]:
# change strings in ordinal data to numbers 

from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder ( 
    categories = [
        ['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'],
        ['< 12 Years', '12 Years', 'Some College', 'College Graduate' ],
        ['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'],
     ]
)

raw_df_encoded_ordinal = ordinal_enc.fit_transform(raw_df_encoded[ordinal_list])
raw_df_encoded_ordinal = pd.DataFrame(raw_df_encoded_ordinal, raw_df_encoded.index)
#ordinal_enc.categories_

raw_df_encoded['age_group'] = raw_df_encoded_ordinal[0]
raw_df_encoded['education'] = raw_df_encoded_ordinal[1]
raw_df_encoded['income_poverty'] = raw_df_encoded_ordinal[2]

test_df_encoded_ordinal = ordinal_enc.fit_transform(test_df_encoded[ordinal_list])
test_df_encoded_ordinal = pd.DataFrame(test_df_encoded_ordinal, test_df_encoded.index)


test_df_encoded['age_group'] = test_df_encoded_ordinal[0]
test_df_encoded['education'] = test_df_encoded_ordinal[1]
test_df_encoded['income_poverty'] = test_df_encoded_ordinal[2]



In [10]:
## impute missing values with mean

from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='mean')
raw_df_encoded_imputed = imp_mode.fit_transform(raw_df_encoded)

raw_df_encoded_imputed = pd.DataFrame(raw_df_encoded_imputed, columns = raw_df_encoded.columns, index = raw_df_encoded.index)

raw_df_median = imp_mode.fit(raw_df_encoded)
test_df_encoded_imputed = imp_mode.transform(test_df_encoded)

test_df_encoded_imputed = pd.DataFrame(test_df_encoded_imputed, columns = test_df_encoded.columns, index = test_df_encoded.index)


In [11]:
# renaming
raw_df_encoded_imputed_list = raw_df_encoded_imputed.columns.tolist()

renamed_list = []
for column_name in raw_df_encoded_imputed_list:
    column_name = re.sub(pattern=r"\$(\d\d)....", repl="\\1k", string=column_name)
    column_name = re.sub(pattern=r"\<\=|\<\s", repl="less_", string=column_name)
    column_name = re.sub(pattern=r"\>\=|\>\s", repl="gr_", string=column_name)
    column_name = re.sub(pattern=r"(\w)\,\s(\w)", repl="\\1_\\2", string=column_name)
    column_name = re.sub(pattern=r"\s\-\s", repl="_-_", string=column_name)
    column_name = re.sub(pattern=r"(\w)\s+(\w)", repl="\\1_\\2", string=column_name)
    column_name = re.sub(pattern=r"(\d\+)\s", repl="\\1_", string=column_name)
    column_name = re.sub(pattern=r"\_\_", repl="_", string=column_name)
    renamed_list.append(column_name)
    
raw_df_encoded_imputed.columns = renamed_list
test_df_encoded_imputed.columns = renamed_list

In [12]:
print(labels_df['h1n1_vaccine'].value_counts())
print(labels_df['seasonal_vaccine'].value_counts())
pd.crosstab(labels_df['h1n1_vaccine'],labels_df['seasonal_vaccine'] )

0    21033
1     5674
Name: h1n1_vaccine, dtype: int64
0    14272
1    12435
Name: seasonal_vaccine, dtype: int64


seasonal_vaccine,0,1
h1n1_vaccine,,
0,13295,7738
1,977,4697


In [13]:
def one_vacc(df):
    if (df['h1n1_vaccine'] == 0) and (df['seasonal_vaccine'] == 0):
        return 0
    elif (df['h1n1_vaccine'] == 1) and (df['seasonal_vaccine'] == 0):
        return 1
    elif (df['h1n1_vaccine'] == 0) and (df['seasonal_vaccine'] == 1):
        return 2
    else:
        return 3





In [14]:
labels_df['one_column_vaccine'] = labels_df.apply(one_vacc, axis = 1)

In [15]:
print(labels_df['one_column_vaccine'].value_counts())

0    13295
2     7738
3     4697
1      977
Name: one_column_vaccine, dtype: int64


In [16]:
oversample = SMOTE()
raw_df_encoded_imputed_balanced, labels_df_balanced = oversample.fit_resample(raw_df_encoded_imputed, labels_df['one_column_vaccine'])

In [17]:
print(labels_df_balanced.value_counts())
raw_df_encoded_imputed_balanced.shape

3    13295
2    13295
1    13295
0    13295
Name: one_column_vaccine, dtype: int64


(53180, 74)

In [18]:
def reverse_h1n1(df):
    if (df['one_column_vaccine'] == 1) or (df['one_column_vaccine'] == 3) :
        return 1
    else:
        return 0
    
def reverse_seasonal(df):
    if (df['one_column_vaccine'] == 2) or (df['one_column_vaccine'] == 3) :
        return 1
    else:
        return 0

In [19]:
labels_df_balanced_pd = pd.DataFrame(labels_df_balanced, columns = ['one_column_vaccine'])

In [20]:
labels_df_balanced_pd['one_column_vaccine'].shape

(53180,)

In [21]:
labels_df_balanced_pd['h1n1_vaccine_new'] = labels_df_balanced_pd.apply(reverse_h1n1, axis = 1)
labels_df_balanced_pd['seasonal_vaccine_new'] = labels_df_balanced_pd.apply(reverse_seasonal, axis = 1)

labels_df_balanced_pd.head(10)

,one_column_vaccine,h1n1_vaccine_new,seasonal_vaccine_new
0,0,0,0
1,2,0,1
2,0,0,0
3,2,0,1
4,0,0,0
5,0,0,0
6,0,0,0
7,3,1,1
8,0,0,0
9,0,0,0


In [22]:
# check for multi collinarity 

pd.set_option("display.max_rows", None, "display.max_columns", None)
corr_data = raw_df_encoded_imputed_balanced.corr().abs()
sorted_corr_data = corr_data.unstack().sort_values(ascending=False)
ones = corr_data.unstack().sort_values(ascending=False) != 1.0
without_ones = sorted_corr_data[ones]
NaNs = corr_data.unstack().sort_values(ascending=False).notna()

without_ones_and_Na = without_ones[NaNs]
big_corrs = corr_data.unstack().sort_values(ascending=False) > 0.5
without_ones_and_Na[big_corrs]

employment_status_nan                 marital_status_nan                      0.907403
marital_status_nan                    employment_status_nan                   0.907403
rent_or_own_nan                       employment_status_nan                   0.810347
employment_status_nan                 rent_or_own_nan                         0.810347
employment_industry_nan               employment_status_Not_in_Labor_Force    0.809612
employment_status_Not_in_Labor_Force  employment_industry_nan                 0.809612
rent_or_own_nan                       marital_status_nan                      0.787514
marital_status_nan                    rent_or_own_nan                         0.787514
health_worker                         employment_industry_fcxhlnwr            0.743144
employment_industry_fcxhlnwr          health_worker                           0.743144
behavioral_outside_home               behavioral_large_gatherings             0.620156
behavioral_large_gatherings           behav

In [23]:
raw_df_encoded_imputed_balanced_dropped = raw_df_encoded_imputed_balanced.drop(columns = 
                                                                               ['marital_status_nan', 
                                                                                'employment_status_nan',
                                                                                'rent_or_own_nan',
                                                                                'employment_industry_nan'])


In [24]:
test_df_encoded_imputed_dropped = test_df_encoded_imputed.drop(columns = ['marital_status_nan', 
                                                                                'employment_status_nan',
                                                                                'rent_or_own_nan',
                                                                                'employment_industry_nan'])

In [25]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
corr_data = raw_df_encoded_imputed_balanced_dropped.corr().abs()
sorted_corr_data = corr_data.unstack().sort_values(ascending=False)
ones = corr_data.unstack().sort_values(ascending=False) != 1.0
without_ones = sorted_corr_data[ones]
NaNs = corr_data.unstack().sort_values(ascending=False).notna()

without_ones_and_Na = without_ones[NaNs]
big_corrs = corr_data.unstack().sort_values(ascending=False) > 0.5
without_ones_and_Na[big_corrs]

health_worker                 employment_industry_fcxhlnwr    0.743144
employment_industry_fcxhlnwr  health_worker                   0.743144
behavioral_outside_home       behavioral_large_gatherings     0.620156
behavioral_large_gatherings   behavioral_outside_home         0.620156
doctor_recc_h1n1              doctor_recc_seasonal            0.568222
doctor_recc_seasonal          doctor_recc_h1n1                0.568222
race_White                    race_Hispanic                   0.563850
race_Hispanic                 race_White                      0.563850
opinion_h1n1_risk             opinion_seas_risk               0.541451
opinion_seas_risk             opinion_h1n1_risk               0.541451
dtype: float64

In [26]:
labels_df_balanced_pd_dropped = labels_df_balanced_pd.drop(columns = ['one_column_vaccine'])
print(labels_df_balanced_pd_dropped.shape)
print(raw_df_encoded_imputed_balanced_dropped.shape)

(53180, 2)
(53180, 70)


In [27]:
labels_df_balanced_pd_dropped.index.rename('respondent_id', inplace = True)
raw_df_encoded_imputed_balanced_dropped.index.rename('respondent_id', inplace = True)
test_df_encoded_imputed_dropped.index.rename('respondent_id', inplace = True)

In [28]:
data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"

labels_df_balanced_pd_dropped.to_csv (data_path_preprocessed / 
    'training_set_labels__balanced.csv')
raw_df_encoded_imputed_balanced_dropped.to_csv (data_path_preprocessed / 
    'training_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_dropped.csv')
test_df_encoded_imputed_dropped.to_csv (data_path_preprocessed / 
    'test_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_dropped.csv')

In [29]:
raw_df_encoded_imputed_balanced.index.rename('respondent_id', inplace = True)
test_df_encoded_imputed.index.rename('respondent_id', inplace = True)
raw_df_encoded_imputed_balanced.to_csv (data_path_preprocessed / 
    'training_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced.csv')
test_df_encoded_imputed.to_csv (data_path_preprocessed / 
    'test_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced.csv')

In [30]:
#standardize dataset

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
raw_df_encoded_imputed_balanced_dropped_stand = scaler.fit_transform(raw_df_encoded_imputed_balanced_dropped)
raw_df_encoded_imputed_balanced_dropped_stand = pd.DataFrame(raw_df_encoded_imputed_balanced_dropped_stand, columns = raw_df_encoded_imputed_balanced_dropped.columns, index = raw_df_encoded_imputed_balanced_dropped.index)

test_df_encoded_imputed_dropped_stand = scaler.fit_transform(test_df_encoded_imputed_dropped)
test_df_encoded_imputed_dropped_stand = pd.DataFrame(test_df_encoded_imputed_dropped_stand, columns = test_df_encoded_imputed_dropped.columns, index = test_df_encoded_imputed_dropped.index)

In [31]:
raw_df_encoded_imputed_balanced_dropped_stand.to_csv (data_path_preprocessed / 
    'training_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_dropped_stand.csv')
test_df_encoded_imputed_dropped_stand.to_csv (data_path_preprocessed / 
    'test_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_dropped_stand.csv')

In [33]:
raw_df_encoded_imputed_balanced_stand = scaler.fit_transform(raw_df_encoded_imputed_balanced)
raw_df_encoded_imputed_balanced_stand = pd.DataFrame(raw_df_encoded_imputed_balanced_stand, columns = raw_df_encoded_imputed_balanced.columns, index = raw_df_encoded_imputed_balanced.index)

test_df_encoded_imputed_stand = scaler.fit_transform(test_df_encoded_imputed)
test_df_encoded_imputed_stand = pd.DataFrame(test_df_encoded_imputed_stand, columns = test_df_encoded_imputed.columns, index = test_df_encoded_imputed.index)

In [34]:
raw_df_encoded_imputed_balanced.index.rename('respondent_id', inplace = True)
test_df_encoded_imputed.index.rename('respondent_id', inplace = True)
raw_df_encoded_imputed_balanced_stand.to_csv (data_path_preprocessed / 
    'training_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_stand.csv')
test_df_encoded_imputed_stand.to_csv (data_path_preprocessed / 
    'test_set_features__Preprocessing_nominal_ordinal_WOE_Impute_balanced_stand.csv')